# **IBL - Processed Behavioral Data**

This tutorial shows how to access data from <[DANDI:00XXXX](https://dandiarchive.org/dandiset/00XXXX/draft)> for the IBL datasets

## Study Overview

[TODO add description]

## Contents

1. [Setup and Data Access](#setup)
2. [Session and Subject Metadata](#metadata)
3. [Trials](#trials)
4. [Pupil Tracking](#pupil)
5. [ROI motion energy](#RME)
6. [Wheel motion](#wheel)
7. [Lick Times](#licks)

---

# 1. Setup and Data Access <a id="setup"></a>

## Import Required Libraries

In [1]:
# Core data manipulation and analysis
from pathlib import Path

import h5py

# Visualization
import matplotlib.pyplot as plt
import remfile
from dandi.dandiapi import DandiAPIClient

# NWB and DANDI access
from pynwb import NWBHDF5IO

# Configure matplotlib
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## Data Access Functions

In [2]:
def load_nwb_from_dandi(dandiset_id, subject_id, session_id):
    """
    Load NWB file from DANDI Archive via streaming.
    """
    pattern = f"sub-{subject_id}/sub-{subject_id}_ses-{session_id}*.nwb"
    
    with DandiAPIClient() as client:
        client.dandi_authenticate()
        assets = client.get_dandiset(dandiset_id, "draft").get_assets_by_glob(
            pattern=pattern, order="path"
        )
        
        s3_urls = []
        for asset in assets:
            s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
            s3_urls.append(s3_url)
        
        if len(s3_urls) != 1:
            raise ValueError(f"Expected 1 file, found {len(s3_urls)} for pattern {pattern}")
        
        s3_url = s3_urls[0]
    
    file = remfile.File(s3_url)
    h5_file = h5py.File(file, "r")
    io = NWBHDF5IO(file=h5_file, load_namespaces=True)
    nwbfile = io.read()
    
    return nwbfile, io


def load_nwb_local(directory_path, subject_id, session_id):
    """
    Load NWB file from local directory.
    """
    directory_path = Path(directory_path)
    nwbfile_path = directory_path / f"sub-{subject_id}_ses-{session_id}.nwb"
    
    if not nwbfile_path.exists():
        raise FileNotFoundError(f"NWB file not found: {nwbfile_path}")
    
    io = NWBHDF5IO(path=nwbfile_path, load_namespaces=True)
    nwbfile = io.read()
    
    return nwbfile, io

---

# 2. Session and Subject Metadata <a id="metadata"></a>

In [3]:
# Load session data
dandiset_id = "00XXXX" #TODO Replace with actual DANDI dandiset ID
subject_id = "SP061"  # Example subject
session_id = "5ce2e17e-8471-42d4-8a16-21949710b328"  # EID for the session
# session_id = "42d7e11e-3185-4a79-a6ad-bbaf47366db2"  # EID for the session


# Choose data source (DANDI streaming or local)
USE_DANDI = False  # Set to False to use local files

if USE_DANDI:
    nwbfile, io = load_nwb_from_dandi(dandiset_id, subject_id, session_id)
else:
    # Specify your local directory path
    local_directory = "E:/ibl_mesoscope_conversion_nwb/processed/nwb_stub"  # Replace with actual path
    nwbfile, io = load_nwb_local(local_directory, subject_id, session_id)

print("=== SESSION INFORMATION ===")
print(f"Experiment description:\n {nwbfile.experiment_description}")
print(f"Session description:\n {nwbfile.session_description}")
print(f"Session start time:\n {nwbfile.session_start_time}")

=== SESSION INFORMATION ===
Experiment description:
 None
Session description:
 A rich text description of the experiment. Can also just be the abstract of the publication.
Session start time:
 2020-01-01 00:00:00-05:00


In [4]:
print("=== SUBJECT INFORMATION ===")
print(f"ID: {nwbfile.subject.subject_id}")
print(f"Age: {nwbfile.subject.age}")
print(f"Strain: {nwbfile.subject.species}")
print(f"Genotype: {nwbfile.subject.genotype}")
print(f"Sex: {nwbfile.subject.sex}")

=== SUBJECT INFORMATION ===
ID: SP061
Age: TBD
Strain: Mus musculus
Genotype: None
Sex: U


---

# 3. Trials <a id="trials"></a>

### TODO add description of IBL trials

In [10]:
print("=== INTERVALS MODULE ===\n")
trials = nwbfile.intervals["trials"]
print("-" * 100)
print("Trials Table (first 5 rows):")
display(trials.to_dataframe().head(5))  # Print first 5 rows as example
print("-" * 100)

=== INTERVALS MODULE ===

----------------------------------------------------------------------------------------------------
Trials Table (first 5 rows):


,start_time,stop_time,choice,feedback_type,reward_volume,contrast_left,contrast_right,probability_left,feedback_time,response_time,stim_on_time,go_cue_time,first_movement_time
id,,,,,,,,,,,,,
0,26.539974,32.715040,-1.0,1.0,1.5,NaN,0.0625,0.5,31.078,31.064014,28.316,27.095,27.747
1,32.746440,40.698956,-1.0,1.0,1.5,NaN,1.0000,0.5,39.060,39.047513,38.583,37.521,38.773
2,40.741550,45.015042,1.0,1.0,1.5,0.125,NaN,0.5,43.373,43.360607,42.882,41.835,42.102
3,45.043743,48.681989,-1.0,1.0,1.5,NaN,0.1250,0.5,47.018,47.005804,46.582,45.524,46.792
4,48.708485,52.232039,1.0,1.0,1.5,0.125,NaN,0.5,50.589,50.576933,50.250,49.174,50.206


----------------------------------------------------------------------------------------------------


TODO:
1. ADD VISUALIZATIONS 

---

# 4. Pupil Tracking <a id="pupil"></a>

In [23]:
print("=== CAMERA PROCESSING MODULE ===\n")
pupil_data_interfaces_names = []
for name, proc in nwbfile.processing["camera"].items():
    if "PupilTracking" not in name:
        continue
    print("-" * 100)
    print(f"{name}: ")
    print("-" * 100)
    for ts_name, ts in proc.time_series.items():
        print(f"\t{ts_name} - {ts.description}")
    pupil_data_interfaces_names.append(name)

=== CAMERA PROCESSING MODULE ===

----------------------------------------------------------------------------------------------------
LeftPupilTracking: 
----------------------------------------------------------------------------------------------------
	LeftRawPupilDiameter - Estimates pupil diameter by taking the median of different computations. The two most straightforward estimates are d1 = top - bottom, d2 = left - right. In addition, assume the pupil is a circle and estimate diameter from other pairs of points.

	LeftSmoothedPupilDiameter - Smoothed and interpolated version of the RawPupilDiameter.
----------------------------------------------------------------------------------------------------
RightPupilTracking: 
----------------------------------------------------------------------------------------------------
	RightRawPupilDiameter - Estimates pupil diameter by taking the median of different computations. The two most straightforward estimates are d1 = top - bottom, d2

TODO:
1. ADD VISUALIZATIONS 

---

# 5. ROI motion energy <a id="RME"></a>

### TODO add description

In [26]:
print("=== CAMERA PROCESSING MODULE ===\n")
motion_energy_series = []
for name, proc in nwbfile.processing["camera"].items():
    if "MotionEnergy" not in name:
        continue
    print("-" * 100)
    print(f"{name} - {proc.description}: ")
    print("-" * 100)
    motion_energy_series.append(name)

=== CAMERA PROCESSING MODULE ===

----------------------------------------------------------------------------------------------------
LeftCameraMotionEnergy - Motion energy calculated for a region of the left camera video that is 153 pixels wide, 102 pixels tall, and the top-left corner of the region is the pixel (194, 368).

CAUTION: As each software will load the video in a different orientation, the ROI might need to be adapted. For example, when loading the video with cv2 in Python, x and y axes are flipped from the convention used above. The region then becomes [368:470, 194:347].: 
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
RightCameraMotionEnergy - Motion energy calculated for a region of the right camera video that is 78 pixels wide, 52 pixels tall, and the top-left corner of the region is the pixel (354, 122).

CAUTION:

TODO:
1. ADD VISUALIZATIONS 

---

# 6. Wheel motion <a id="wheel"></a>

### TODO add description

In [35]:
print("=== WHEEL PROCESSING MODULE ===\n")
motion_energy_series = []
for name, proc in nwbfile.processing["wheel"].items():
    if "CompassDirection" in name:
        print("-" * 100)
        print(f"{name}: ")
        print("-" * 100)
        for ss_name, ss in proc.spatial_series.items():
            print(f"\t{ss_name} - {ss.description}")   
    elif "Intervals" in name:
        print("-" * 100)
        print(f"{name} - {proc.description}: ")
        display(proc.to_dataframe().head(5))  # Display first 5 rows
        print("-" * 100)
    else:
        print("-" * 100)
        print(f"{name} - {proc.description}: ")
        print("-" * 100)
        motion_energy_series.append(name)

=== WHEEL PROCESSING MODULE ===

----------------------------------------------------------------------------------------------------
CompassDirection: 
----------------------------------------------------------------------------------------------------
	WheelPositionSeries - Absolute unwrapped angle of the wheel from session start. The sign from the subject perspective corresponds to mathematical convention; counter-clockwise is positive. The wheel diameter is 6.2 cm and the number of ticks is 4096 per revolution.

----------------------------------------------------------------------------------------------------
WheelAcceleration - The acceleration estimated from the position interpolated at a frequency of 1000 Hz and passed through an 8th order lowpass Butterworth filter.: 
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
WheelVelo

,start_time,stop_time,peak_amplitude
id,,,
0,17.569,17.719,-0.019558
1,20.851,21.015,-0.013635
2,21.726,21.858,-0.011853
3,22.091,22.213,-0.021135
4,22.981,24.308,0.336964


----------------------------------------------------------------------------------------------------


TODO:
1. ADD VISUALIZATIONS 

---

# 7. Lick Times <a id="licks"></a>

### TODO add description

In [36]:
print("=== CAMERA PROCESSING MODULE ===\n")

for name, proc in nwbfile.processing["camera"].items():
    if "LickTimes" in name:
        break
print("-" * 100)
print(f"{name} - {proc.description}: ")
display(proc.to_dataframe().head(5))  # Display first 5 rows
print("-" * 100)

=== CAMERA PROCESSING MODULE ===

----------------------------------------------------------------------------------------------------
LickTimes - Time stamps of licks as detected from tongue dlc traces. If left and right camera exist, the licks detected from both cameras are combined.: 


,lick_time
id,
0,13.644
1,13.661
2,13.728
3,13.744
4,13.761


----------------------------------------------------------------------------------------------------


TODO:
1. ADD VISUALIZATIONS 